In [134]:
import functools
import logging
from dataclasses import dataclass
import random
import numpy as np
from tqdm.auto import tqdm
from icecream import ic
import pandas as pd
from itertools import permutations
import pandas as pd
from geopy.distance import geodesic

logging.basicConfig(level=logging.INFO)

In [135]:
data = pd.read_csv('cities/vanuatu.csv', header=None , names=['City', 'x', 'y'])
# cities #to print the data of all the cities
cities = data[['x', 'y']].values
city_name = data['City'].values
country_coords = {row['City']: (row['x'], row['y']) for _, row in data.iterrows()}

In [ ]:
def geodesic_distance(city1, city2):
    return geodesic(city1, city2).kilometers

def euclidean_distance(city1, city2):
    return np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

def calculate_total_distance(tour, country_coords, countries=None):
    total_distance = 0
    for i in range(len(tour) - 1):
        city1, city2 = tour[i], tour[i + 1]
        total_distance += geodesic_distance(country_coords[city1], country_coords[city2])
    total_distance += geodesic_distance(country_coords[tour[-1]], country_coords[tour[0]])
    return total_distance



## Algorithm Brute Force TSP

In [138]:
def tsp_bruteforce(country_coords):
    countries = list(country_coords.keys())
    min_distance = float('inf')
    optimal_tour = None
    step_counter = 0
    for perm in permutations(countries):
        current_distance = calculate_total_distance(perm, country_coords, countries)
        step_counter += 1
        if current_distance < min_distance:
            min_distance = current_distance
            optimal_tour = perm

    logging.info(f"Brute Force Steps: {step_counter}")        
    return optimal_tour, min_distance, step_counter

In [ ]:
# results from the bruth force
_, brute_distance, brute_steps = tsp_bruteforce(country_coords)
logging.info(f"Brute Force Distance: {brute_distance} km, Steps: {brute_steps}")


# Genetic Algorithm

In [139]:
#first creatation of the intial population
def create_initial_population(pop_size, num_cities):
    return [random.sample(range(num_cities), num_cities) for _ in range(pop_size)]

#the funtion which can be used for the selection
def selection(population, fitness, num_parents):
    fitness_sorted = sorted(zip(population, fitness), key=lambda x: x[1])
    selected = [individual for individual, _ in fitness_sorted[:num_parents]]
    return selected

#for the crossover
def crossover(p1,p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    pointer = end
    for gene in p2:
        if gene not in child:
            if pointer >=size:
                pointer = 0
            child[pointer] = gene
            pointer += 1
    return child

#swapping of children using mutation
def mutation(tour, mutation_rate = 0.01):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
    return tour


In [140]:
def genetic_algorithm(countries, country_coords, population_size=100, generations=500, mutation_rate=0.01, num_parents=20):
    num_cities = len(countries)
    population = create_initial_population(population_size, num_cities)
    step_counter =0 
    for generation in tqdm(range(generations)):
        # Calculate fitness of each individual
        fitness = [calculate_total_distance([city_name[i] for i in tour], country_coords, city_name) for tour in population]
        
        # Selection
        parents = selection(population, fitness, num_parents)
        
        # Generate new population through crossover and mutation
        next_population = []
        while len(next_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutation(child, mutation_rate)
            next_population.append(child)
        
        population = next_population
        step_counter +=1
        # Print best solution every 50 generations
        if generation % 50 == 0:
            best_distance = min(fitness)
            print(f"Generation {generation}, Best Distance: {best_distance:.2f} km")
    
    
    # final_fitness = [calculate_total_distance([city_name[i] for i in tour], country_coords) for tour in population]
    # best_tour_index = np.argmin(final_fitness)
    best_tour = population[np.argmin(fitness)]

    best_distance = min(fitness)

    # Convert indices back to country names
    best_tour_cities = [city_name[i] for i in best_tour]
    return best_tour_cities, best_distance, step_counter

In [ ]:
# output of genetic algorithm
_, ga_distance, ga_steps = genetic_algorithm(city_name, country_coords)
logging.info(f"Genetic Algorithm Distance: {ga_distance} km, Steps: {ga_steps}")


## Nearest Neighbor Algorithm

In [141]:
def tsp_nearest_neighbor(cities):
    n = len(cities)
    visited = [False] * n
    current_city = 0
    visited[current_city] = True
    tour = [current_city]
    total_cost = 0
    step_counter = 0
    for _ in range(n - 1):
        next_city = None
        min_dist = float('inf')
        for i in range(n):
            if not visited[i]:
                dist = euclidean_distance(cities[current_city], cities[i])
                if dist < min_dist:
                    min_dist = dist
                    next_city = i
        tour.append(next_city)
        total_cost += min_dist
        visited[next_city] = True
        current_city = next_city
        step_counter += 1
    total_cost += euclidean_distance(cities[current_city], cities[tour[0]])
    tour.append(tour[0])

    logging.info(f"Nearest Neighbor Steps: {step_counter}")
    return tour, total_cost, step_counter

In [ ]:
# # tsp neighbour outputs
_, nn_distance, nn_steps = tsp_nearest_neighbor(cities)
logging.info(f"Nearest Neighbor Distance: {nn_distance*100} km, Steps: {nn_steps}")

## 2-Opt Optimization 

In [142]:
# 2-opt code
def two_opt(tour, cities):
    best_distance = calculate_total_distance(tour, cities)
    best_tour = tour[:]
    improved = True
    step_counter = 0

    while improved:
        improved = False
        for i in range(1, len(tour) - 2):
            for j in range(i + 1, len(tour)):
                if j - i == 1: 
                    continue  # skip adjacent nodes
                new_tour = best_tour[:]
                new_tour[i:j] = tour[j - 1:i - 1:-1]  # reverse the tour section
                new_distance = calculate_total_distance(new_tour, cities)
                
                step_counter +=1
                if new_distance < best_distance:
                    best_tour = new_tour
                    best_distance = new_distance
                    improved = True
        tour = best_tour
    logging.info(f"2-Opt Steps: {step_counter}")
    return best_tour, best_distance, step_counter

In [ ]:
# Improve tour with 2-Opt
optimized_tour, opt_distance, opt_steps = two_opt([0, 1, 2, 3, 4, 5, 6, 7], cities)  # Example initial tour
logging.info(f"2-Opt Distance: {opt_distance} km, Steps: {opt_steps}")

INFO:root:2-Opt Steps: 30
INFO:root:2-Opt Distance: 1343.9036980299845 km, Steps: 30
